In [231]:
using PyCall
using Distances
using StatsBase
using LinearAlgebra
using JuMP
using Gurobi
using CSV
using Distances
using PyPlot
using SparseArrays
using Printf

In [243]:
py"""
import numpy as np
import cv2


class RootSIFT:
    def __init__(self):
        self.extractor = cv2.xfeatures2d.SIFT_create()

    def compute(self, image, kps, eps=1e-7):
        (kps, descs) = self.extractor.compute(image, kps)
        if len(kps) == 0:
            return ([], None)

        descs /= (descs.sum(axis=1, keepdims=True) + eps)
        descs = np.sqrt(descs)
        return (kps, descs)
def rootSIFT(img_path):
    image = cv2.imread(img_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.xfeatures2d.SIFT_create()
    (kps, descs) = sift.detectAndCompute(gray, None)
    rs = RootSIFT()
    (kps, descs) = rs.compute(gray, kps)

    return descs

"""

img1_path = "../data\\test\\test_1.jpg"
img2_path = "../data\\test\\test_2.jpg"

descs1 = py"rootSIFT"(img1_path);
descs2 = py"rootSIFT"(img2_path);

In [260]:
cost = pairwise(Euclidean(), descs1, descs2; dims=1)

36×37 Array{Float32,2}:
 1.03707   0.758385  0.954285  0.897025  …  0.82175   0.900018  0.870877
 1.06751   0.812301  0.948864  0.934764     0.896925  0.887612  0.80029 
 0.911025  0.747179  0.742196  1.03903      0.818866  0.823335  0.910128
 0.985357  1.01972   1.01152   0.957808     0.906323  0.937984  0.954617
 0.780923  0.974776  0.949937  1.03064      1.02854   1.03817   1.01997 
 0.932993  0.832282  0.773582  0.684676  …  0.842932  0.782594  0.942967
 0.873618  1.00487   1.02046   1.16965      0.894259  0.988821  0.98403 
 1.01373   0.758578  0.743826  0.952549     0.79761   0.749669  0.903804
 1.03989   0.736468  0.699093  0.929248     0.764214  0.716332  0.926073
 1.0574    0.854865  1.00997   0.948132     0.906436  0.949825  0.956225
 1.03778   0.973754  0.943744  0.84878   …  0.875354  0.786158  0.901227
 0.861043  0.936131  0.859931  0.769906     0.873672  0.667005  0.826686
 1.00314   0.930922  0.946456  0.874922     0.920634  0.814508  0.66734 
 ⋮                         

In [235]:
P = ones(size(descs1,1));
Q = ones(size(descs2,1));

In [236]:
solCount = 1000
m = JuMP.direct_model(Gurobi.Optimizer(PoolSearchMode=2, PoolSolutions=solCount, SolutionNumber=0,PoolGap = 0.0001))

@variable(m, X[axes(cost,1), axes(cost,2)] ≥ 0, Int)
@objective(m, Min, cost ⋅ X)
@constraint(m,sum(X) .== min(sum(P), sum(Q)))
@constraint(m, X * ones(Int, length(Q)) .<= P)
@constraint(m, X'ones(Int, length(P)) .<= Q);
optimize!(m)
solution_pool = zeros(solCount, length(P),length(Q))
cnt = 0
for i in 0:(solCount-1)
    try
        setparam!(m.moi_backend.inner,"SolutionNumber", i)
        
        xn = Gurobi.get_dblattrarray(m.moi_backend.inner, "Xn", 1, length(X))
        xn_val = Gurobi.get_dblattr(m.moi_backend.inner, "PoolObjVal")
        
        default = zeros(length(P),length(Q))
        for i in 0:length(P)-1
            default[i+1,:] = xn[(i*length(Q))+1:(i+1)*length(Q)]
        end
        solution_pool[i+1,:,:] = default
        cnt+=1
    catch 
        break
    end
end
sol_pool = deepcopy(solution_pool[1:cnt,:,:]);

Optimize a model with 73 rows, 1296 columns and 3888 nonzeros
Variable types: 0 continuous, 1296 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-04, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+01]
Presolve time: 0.00s
Presolved: 73 rows, 1296 columns, 3888 nonzeros
Variable types: 0 continuous, 1296 integer (1296 binary)
Found heuristic solution: objective 0.0013218

Root relaxation: objective 1.321829e-03, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0          -    0         0.00132    0.00132  0.00%     -    0s
Optimal solution found at node 0 - now completing solution pool...
     0     0          -    0         0.00132    0.00132  0.00%     -    0s
     0     0          -    0         0.00132    0.00132  0.00%     -    0s
     0     2          -    0         0.